In [1]:
import pandas as pd

import techminer.data as data

filepath = "../data/papers/urban-agriculture.csv"
df = pd.read_csv(filepath)
df = data.load_scopus(df)

2020-06-04 16:53:47,324 - INFO - Renaming and selecting columns ...
2020-06-04 16:53:53,011 - INFO - Formatting author names ...
2020-06-04 16:53:53,015 - INFO - Disambiguating author names ...
2020-06-04 16:53:53,074 - INFO - Removing part of titles in foreing languages ...
2020-06-04 16:53:53,077 - INFO - Fusioning author and index keywords ...
2020-06-04 16:53:53,090 - INFO - NumExpr defaulting to 8 threads.
2020-06-04 16:53:53,105 - INFO - Extracting countries from affiliations ...
2020-06-04 16:53:59,229 - INFO - Extracting institutions from affiliations ...
2020-06-04 16:53:59,239 - INFO - Extracting country of 1st author ...
2020-06-04 16:53:59,243 - INFO - Extracting affiliation of 1st author ...
2020-06-04 16:53:59,247 - INFO - Counting number of authors ...


In [ ]:
data.coverage(df)

### Keywords

### FindString Dialog Box pag. 69

### Cleaning a list pag. 102

### Fuzzy match editor pag.126

### Stemming And / Stemming Or

### APPS

In [3]:
import techminer.by_year as year
year.app(df)

AppLayout(children=(HTML(value="<h1>Summary by Year</h1><hr style='height:2px;border-width:0;color:gray;backgr…

In [4]:
import techminer.by_term as term
term.app(df)

AppLayout(children=(HTML(value="<h1>Summary by Term</h1><hr style='height:2px;border-width:0;color:gray;backgr…

In [5]:
import techminer.by_term_per_year as term_year
term_year.app(df)

AppLayout(children=(HTML(value="<h1>Summary by Term per Year</h1><hr style='height:2px;border-width:0;color:gr…

In [6]:
import techminer.co_occurrence as co_occurrence
co_occurrence.app(df)

AppLayout(children=(HTML(value="<h1>Co-occurrence Analysis</h1><hr style='height:2px;border-width:0;color:gray…

In [ ]:
import ipywidgets as widgets
from ipywidgets import Layout, AppLayout

In [ ]:

COLUMNS = [
    "Author Keywords",
    "Authors",
    "Countries",
    "Country 1st",
    "Document type",
    "Index Keywords",
    "Institution 1st",
    "Institutions",
    "Keywords",
    "Source title",
]

In [ ]:
FIGSIZE = (14, 10.0) ## Co-occurrente-analysis  <---- no tiene figura
FIGSIZE = (14, 9.0)

# NUEVA INTERFAZ

In [ ]:
from techminer.summary_by_term_per_year import documents_by_term_per_year, citations_by_term_per_year

In [ ]:
import techminer.plots as plt
from techminer.plots import COLORMAPS
from techminer.summary_by_term import citations_by_term, documents_by_term

WIDGET_WIDTH = "200px"
LEFT_PANEL_HEIGHT = "588px"
RIGHT_PANEL_WIDTH = "870px"
FIGSIZE = (14, 10.0)
PANE_HEIGHTS = ["80px", "650px", 0]

COLUMNS = [
    "Author Keywords",
    "Authors",
    "Countries",
    "Country 1st",
    "Document type",
    "Index Keywords",
    "Institution 1st",
    "Institutions",
    "Keywords",
    "Source title",
]



In [ ]:
from techminer.correlation import corr


def body_0(x):
    #
    def server(**kwargs):
        #
        # Logic
        #
        column = kwargs["term"]
        by = kwargs["by"]
        method = kwargs["method"]
        min_link_value = kwargs["min_link_value"]
        cmap = kwargs["cmap"]
        #
        matrix = corr(
                x,
                column=column,
                by=by,
                method=method,
                min_link_value=min_link_value,
                cmap=cmap,
                as_matrix=True,
                filter_by=None,
            )
        #
        output.clear_output()
        with output:
            if len(matrix.columns) < 51 and len(matrix.index) < 51:
                display(matrix.style.background_gradient(cmap=cmap))
            else:
                display(matrix)
                
    #
    # UI
    #
    controls = [
        {
            "arg": "term",
            "desc": "Term to analyze:",
            "widget": widgets.Select(
                options=[z for z in COLUMNS if z in x.columns],
                ensure_option=True,
                disabled=False,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        {
            "arg": "by",
            "desc": "By Term:",
            "widget": widgets.Select(
                options=[z for z in COLUMNS if z in x.columns],
                ensure_option=True,
                disabled=False,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        {
            "arg": "method",
            "desc": "Method:",
            "widget": widgets.Select(
                options=['pearson', 'kendall', 'spearman'],
                ensure_option=True,
                disabled=False,
                continuous_update=True,
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        {
            "arg": "cmap",
            "desc": "Colormap:",
            "widget": widgets.Dropdown(
                options=COLORMAPS, disable=False, layout=Layout(width=WIDGET_WIDTH),
            ),
        },
        {
            "arg": "min_link_value",
            "desc": "Min link value:",
            "widget": widgets.FloatSlider(
                value=-1,
                min=-1,
                max=1.0,
                step=0.01,
                disabled=False,
                continuous_update=True,
                orientation='horizontal',
                readout=True,
                readout_format='.1f',
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },                    
        {
            "arg": "top_n",
            "desc": "Top N:",
            "widget": widgets.IntSlider(
                value=10,
                min=10,
                max=50,
                step=1,
                disabled=False,
                continuous_update=False,
                orientation="horizontal",
                readout=True,
                readout_format="d",
                layout=Layout(width=WIDGET_WIDTH),
            ),
        },
    ]
    #
    args = {control["arg"]: control["widget"] for control in controls}
    output = widgets.Output()
    with output:
        display(widgets.interactive_output(server, args,))
    return widgets.HBox(
        [
            widgets.VBox(
                [
                    widgets.VBox(
                        [widgets.Label(value=control["desc"]), control["widget"]]
                    )
                    for control in controls
                ],
                layout=Layout(height=LEFT_PANEL_HEIGHT, border="1px solid gray"),
            ),
            widgets.VBox([output], layout=Layout(width=RIGHT_PANEL_WIDTH)),
        ]
    )


def app(df):
    #
    body = widgets.Tab()
    body.children = [body_0(df)]
    body.set_title(0, "Matrix")
    #
    return AppLayout(
        header=widgets.HTML(
            value="<h1>{}</h1><hr style='height:2px;border-width:0;color:gray;background-color:gray'>".format(
                "Correlation Analysis"
            )
        ),
        center=body,
        pane_heights=PANE_HEIGHTS,
    )


#####
app(df)